In [471]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [ ]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('evaluate_all.csv')

df = df.drop(columns=['pbid'])
df = df[df['ARG'] <= 100000]
grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean',
    'classcial':'mean',
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=["ARG",'classcial'])
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'NewSolver', 'QTOSimplifySolver', 'QTOSimplifyDiscardSolver']
method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'QTOSimplifyDiscardSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([["ARG",'classcial'], method_order]))

pivot_df

ARG                            \
                             HeaSolver PenaltySolver ChocoSolver   
pkid variables constraints                                         
0    6         3             56.078727     49.814650    0.106678   
1    15        8            145.390764    105.645066    0.446079   

                                                      classcial                \
                           QTOSimplifyDiscardSolver   HeaSolver PenaltySolver   
pkid variables constraints                                                      
0    6         3                           0.005745   18.671726      8.150127   
1    15        8                           0.082184  307.268225    119.331323   

                                                                 
                           ChocoSolver QTOSimplifyDiscardSolver  
pkid variables constraints                                       
0    6         3              0.610565                 0.214183  
1    15        8              2.979041                 3.856116

In [473]:
# 选取 'depth' 下的所有列，并剔除 'NewSolver' 和 'QTOSimplifySolver'
df_1 = pivot_df['best_solution_probs'].drop(columns=['NewSolver', 'QTOSimplifySolver'], errors='ignore')

# 选取 'culled_depth_per_param' 下的 'QTOSimplifyDiscardSolver' 列
df_2 = pivot_df['ARG'].drop(columns=['NewSolver', 'QTOSimplifySolver'], errors='ignore')

# 合并处理后的 'depth' 列与 'QTOSimplifyDiscardSolver' 列，并指定后缀避免冲突
culled_depth_df = df_1.join(df_2, lsuffix='_1', rsuffix='_2')

# 按行遍历输出值，每个值用 & 分隔
for row in culled_depth_df.itertuples(index=False):
    formatted_row = [f"{round(x, 2):5.02f}" for x in row]
    print(" & ".join(formatted_row))


KeyError: 'best_solution_probs'

In [ ]:
# 提取所需列
best_solution_probs_qto_discard = pivot_df['best_solution_probs']['QTOSimplifyDiscardSolver']
best_solution_probs_choco = pivot_df['best_solution_probs']['ChocoSolver']
arg_qto_discard = pivot_df['ARG']['QTOSimplifyDiscardSolver']
arg_choco = pivot_df['ARG']['ChocoSolver']

# 方法1：每行相除取平均
# best_solution_probs 下 QTOSimplifyDiscardSolver / ChocoSolver 的倍数
row_wise_ratio_best_solution_probs = (best_solution_probs_qto_discard / best_solution_probs_choco).mean()

# ARG 下 ChocoSolver / QTOSimplifyDiscardSolver 的倍数
row_wise_ratio_arg = (arg_choco / arg_qto_discard).mean()

# 方法2：整列取平均后相除
# best_solution_probs 下 QTOSimplifyDiscardSolver / ChocoSolver 的整列平均的倍数
col_avg_ratio_best_solution_probs = best_solution_probs_qto_discard.mean() / best_solution_probs_choco.mean()

# ARG 下 ChocoSolver / QTOSimplifyDiscardSolver 的整列平均的倍数
col_avg_ratio_arg = arg_choco.mean() / arg_qto_discard.mean()

# 输出结果
print(f"行平均 - QTOSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {row_wise_ratio_best_solution_probs:.2f}")
print(f"行平均 - ChocoSolver / QTOSimplifyDiscardSolver (ARG): {row_wise_ratio_arg:.2f}")
print(f"列平均 - QTOSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {col_avg_ratio_best_solution_probs:.2f}")
print(f"列平均 - ChocoSolver / QTOSimplifyDiscardSolver (ARG): {col_avg_ratio_arg:.2f}")


行平均 - QTOSimplifyDiscardSolver / ChocoSolver (best_solution_probs): nan
行平均 - ChocoSolver / QTOSimplifyDiscardSolver (ARG): nan
列平均 - QTOSimplifyDiscardSolver / ChocoSolver (best_solution_probs): nan
列平均 - ChocoSolver / QTOSimplifyDiscardSolver (ARG): nan


In [ ]:

# 读入 CSV 文件
df = pd.read_csv('evaluate_TSP.csv')

df = df.drop(columns=['pbid'])

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean'
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df

FileNotFoundError: [Errno 2] No such file or directory: 'evaluate_TSP.csv'

In [ ]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('eva_SCP.csv')

df = df.drop(columns=['pbid'])

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean'
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df

In [ ]:
# 定义格式化函数
import numpy as np
def format_value(val):
    if np.isnan(val):  # 如果是NaN值则返回空字符
        return "NaN"
    elif val < 100:  # 如果值小于100，保留3位有效数字
        return f"{val:.2f}"
    else:  # 如果值大于等于100，不保留小数点
        return f"{val:.0f}"

# 按行遍历输出，每个值用 & 分隔
for row in pivot_df.itertuples(index=False):
    formatted_row = [format_value(x) for x in row]
    print(" & ".join(formatted_row))

In [ ]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('eva_old.csv')

df = df.drop(columns=['pbid'])

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean'
})
grouped_df
# 假设grouped_df是一个Pandas DataFrame
grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
grouped_df = grouped_df.drop(columns=['layers'])
## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




method_order = ['penalty', 'cyclic', 'HEA', 'commute']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df

In [ ]:
pivot_df['ARG']

In [ ]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('eval_eliminate copy 2.csv')

df = df.drop(columns=['pbid','layers', 'variables', 'constraints'])

grouped_df = df.groupby(['pkid',  'method'], as_index=False).agg({
    "successrate": 'mean',
})
grouped_df
# 假设grouped_df是一个Pandas DataFrame
# grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# grouped_df = grouped_df.drop(columns=['layers'])
## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




method_order = ['penalty', 'cyclic', 'HEA', 'commute']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df